In [1]:
%matplotlib inline
from collections import Counter
from pathlib import Path
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Set folder paths for outputs


In [2]:
import sys
sys.path.append("../../..")

In [3]:
from utils.auxSamplingStudy import *

In [4]:
from analysis.data.ccard.config import DATA, MODELS, REPORTS, idbin, idcat, idnum

# Read the file

In [5]:
df= pd.read_csv(DATA / 'ccard.csv')

### Three columns  are removed from this file- clientnum, Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1 and Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2

In [6]:
df.columns

Index(['Attrition_Flag', 'Customer_Age', 'Gender', 'Dependent_count',
       'Education_Level', 'Marital_Status', 'Income_Category', 'Card_Category',
       'Months_on_book', 'Total_Relationship_Count', 'Months_Inactive_12_mon',
       'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal',
       'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt',
       'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio'],
      dtype='object')

In [7]:
df.rename(columns = {'Attrition_Flag': 'target'},inplace = True)

In [8]:
df.target.unique()

array(['Existing Customer', 'Attrited Customer'], dtype=object)

#### In order to feed the data to any machine learning method, it's convenient to change strings to numeric values. So, we are going to change 'Existing' to 0 and 'Attrited' to 1

In [9]:
df['target'].replace({'Existing Customer': 0, 'Attrited Customer' : 1},inplace = True)


In [10]:
df.drop_duplicates(inplace=True)

In [11]:
df.isnull().sum()

target                      0
Customer_Age                0
Gender                      0
Dependent_count             0
Education_Level             0
Marital_Status              0
Income_Category             0
Card_Category               0
Months_on_book              0
Total_Relationship_Count    0
Months_Inactive_12_mon      0
Contacts_Count_12_mon       0
Credit_Limit                0
Total_Revolving_Bal         0
Avg_Open_To_Buy             0
Total_Amt_Chng_Q4_Q1        0
Total_Trans_Amt             0
Total_Trans_Ct              0
Total_Ct_Chng_Q4_Q1         0
Avg_Utilization_Ratio       0
dtype: int64

In [12]:
df.shape

(10127, 20)

# Remove unknown observations

In [13]:
for cols in ['Education_Level', 'Marital_Status', 'Income_Category']:
    df = df.loc[~df[cols].isin(['Unknown', 'unknown'])]


In [14]:
Counter(df.target)

Counter({0: 5968, 1: 1113})

In [15]:
df.columns

Index(['target', 'Customer_Age', 'Gender', 'Dependent_count',
       'Education_Level', 'Marital_Status', 'Income_Category', 'Card_Category',
       'Months_on_book', 'Total_Relationship_Count', 'Months_Inactive_12_mon',
       'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal',
       'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt',
       'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio'],
      dtype='object')

In [16]:
df.shape

(7081, 20)

In [17]:
df_t1 = df.loc[df['target'] == 1].head(500)
df_t2 = df.loc[df['target'] == 0]
df = df_t1.append(df_t2)

In [18]:
df.shape

(6468, 20)

# Create helper functions

In [19]:
## Replace Male with 0 and Female with 1
df['Gender'].replace({'M': 0, 'F': 1},inplace = True)

In [20]:
Counter(df.target)

Counter({1: 500, 0: 5968})

# Num of minority instances is 500 and majority instances is 5968

In [21]:
df.columns

Index(['target', 'Customer_Age', 'Gender', 'Dependent_count',
       'Education_Level', 'Marital_Status', 'Income_Category', 'Card_Category',
       'Months_on_book', 'Total_Relationship_Count', 'Months_Inactive_12_mon',
       'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal',
       'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt',
       'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio'],
      dtype='object')

In [22]:
# df_feature= df.drop(['target'],1)
# df_target= df[['target']]

In [23]:
df.columns

Index(['target', 'Customer_Age', 'Gender', 'Dependent_count',
       'Education_Level', 'Marital_Status', 'Income_Category', 'Card_Category',
       'Months_on_book', 'Total_Relationship_Count', 'Months_Inactive_12_mon',
       'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal',
       'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt',
       'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio'],
      dtype='object')

In [24]:
encoded_df = MultiColumnLabelEncoder(columns = ['Education_Level', 'Marital_Status', 
                                                'Income_Category', 'Card_Category']).fit_transform(df)


In [25]:
## Check if categorical variables and the target has some correlation

anovap_value = list()
for cols in ['Gender', 'Education_Level', 'Marital_Status', 'Income_Category', 'Card_Category']:
    statistic, p = stats.f_oneway(encoded_df[cols], encoded_df['target'])
    anovap_value.append(tuple([cols, p]))
    
print(anovap_value)

[('Gender', 0.0), ('Education_Level', 0.0), ('Marital_Status', 0.0), ('Income_Category', 0.0), ('Card_Category', 9.292702718255405e-29)]


In [26]:
from scipy.stats import chi2_contingency
chi2p_value = list()
for cols in ['Gender', 'Education_Level', 'Marital_Status', 'Income_Category', 'Card_Category']:
    obs = pd.crosstab(encoded_df[cols], encoded_df['target'])
    g, p, dof, expctd = chi2_contingency(obs)
    chi2p_value.append(tuple([cols, p]))
    
print(chi2p_value)

[('Gender', 0.058843486727238416), ('Education_Level', 0.04097165799983772), ('Marital_Status', 0.12227261153702998), ('Income_Category', 0.23228068911971203), ('Card_Category', 0.5613422371115784)]


In [27]:
encoded_df= encoded_df.drop(['target'],1)
df_target= df[['target']]

In [28]:
1- 2000 / encoded_df.shape[0]

0.6907854050711193

In [29]:
np.sum(df_target)/df_target.shape[0]

target    0.077304
dtype: float64

In [30]:
#Sampling: Added by me
encoded_df, aux1, df_target, aux2 = train_test_split(encoded_df, df_target, test_size=0.70, random_state=12)

In [31]:
df_target.shape

(1940, 1)

In [32]:
np.sum(df_target)

target    160
dtype: int64

In [33]:
np.sum(df_target)/df_target.shape[0]

target    0.082474
dtype: float64

In [34]:
1/12

0.08333333333333333

In [35]:
# Split the raw data into train and test set. Split ratio = 75:25
X_train, X_test, y_train, y_test = train_test_split(encoded_df, df_target, test_size=0.25, random_state=12)

In [36]:
col_list = list(X_train.columns)
X_train.index = pd.RangeIndex(len(X_train.index))
y_train.index = pd.RangeIndex(len(y_train.index))
X_test.index = pd.RangeIndex(len(X_test.index))
y_test.index = pd.RangeIndex(len(y_test.index))

In [37]:
## Apply standard scaler on the features , so that euclidean distance calculation in SMOTE is not biased

#sc = StandardScaler()
#X_train = sc.fit_transform(X_train)
#X_test = sc.transform(X_test)

columns = [0, 2, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
name_columns = X_train.columns[columns]

sc = StandardScaler()
X_train[name_columns] = sc.fit_transform(X_train[name_columns])
X_test[name_columns] = sc.transform(X_test[name_columns])

In [38]:
X_train = pd.DataFrame(X_train, index=range(X_train.shape[0]),
                          columns=col_list)
X_test = pd.DataFrame(X_test, index=range(X_test.shape[0]),
                          columns=col_list)

In [39]:
X_train.to_csv(DATA / "ccard_X_train.csv", index = False)
y_train.to_csv(DATA / "ccard_y_train.csv", index = False)
X_test.to_csv(DATA / "ccard_X_test.csv", index = False)
y_test.to_csv(DATA / "ccard_y_test.csv", index = False)